In [1]:
!pip install dask
!pip install pandas
!pip install psutil

In [2]:
!pip install kaggle

In [33]:
import pandas as pd
import numpy as np
import os

def create_real_5gb_test_file_smart(target_size_gb=5):
    print("🚀 بدء إنشاء ملف تجريبي ~5GB ...")

    file_name = 'real_5gb_data.csv'
    if os.path.exists(file_name):
        os.remove(file_name)

    process = 0
    chunk_num = 0
    rows_per_chunk = 300_000
    text_length1 = 200
    text_length2 = 150

    while True:
        chunk = pd.DataFrame({
            'id': range(chunk_num*rows_per_chunk, (chunk_num+1)*rows_per_chunk),
            'feature1': np.random.randn(rows_per_chunk),
            'feature2': np.random.exponential(2.0, rows_per_chunk),
            'feature3': np.random.uniform(0, 1000, rows_per_chunk),
            'feature4': np.random.randint(0, 100000, rows_per_chunk),
            'feature5': np.random.randn(rows_per_chunk),
            'feature6': np.random.randn(rows_per_chunk),
            'feature7': np.random.randn(rows_per_chunk),
            'feature8': np.random.exponential(1.5, rows_per_chunk),
            'category1': np.random.choice(list('ABCDEFGHIJ'), rows_per_chunk),
            'category2': np.random.choice(['High', 'Medium', 'Low', 'Very_High', 'Very_Low'], rows_per_chunk),
            'text_field1': ['x'*text_length1 for _ in range(rows_per_chunk)],
            'text_field2': ['y'*text_length2 for _ in range(rows_per_chunk)],
            'timestamp': pd.date_range('2020-01-01', periods=rows_per_chunk, freq='s'),
            'boolean_flag': np.random.choice([True, False], rows_per_chunk),
            'numeric_extra1': np.random.randint(0, 50000, rows_per_chunk),
            'numeric_extra2': np.random.randn(rows_per_chunk),
            'numeric_extra3': np.random.uniform(-100,100, rows_per_chunk)
        })

        mode = 'w' if chunk_num == 0 else 'a'
        header = chunk_num == 0
        chunk.to_csv(file_name, mode=mode, header=header, index=False)

        current_size_gb = os.path.getsize(file_name) / (1024**3)
        chunk_num += 1
        print(f"⏳ الدفعة {chunk_num} - الحجم الحالي: {current_size_gb:.2f} GB")

        if current_size_gb >= target_size_gb:
            print("✅ تم الوصول إلى الحجم المطلوب.")
            break

    final_rows = sum(1 for line in open(file_name)) - 1
    print(f"\n📦 الحجم النهائي: {current_size_gb:.2f} GB")
    print(f"📈 عدد الصفوف: {final_rows:,}")
    print(f"🎯 عدد الأعمدة: {len(chunk.columns)}")

    return file_name

# تشغيل الدالة
big_csv_path = create_real_5gb_test_file_smart()


🚀 بدء إنشاء ملف تجريبي ~5GB ...
⏳ الدفعة 1 - الحجم الحالي: 0.16 GB
⏳ الدفعة 2 - الحجم الحالي: 0.32 GB
⏳ الدفعة 3 - الحجم الحالي: 0.48 GB
⏳ الدفعة 4 - الحجم الحالي: 0.65 GB
⏳ الدفعة 5 - الحجم الحالي: 0.81 GB
⏳ الدفعة 6 - الحجم الحالي: 0.97 GB
⏳ الدفعة 7 - الحجم الحالي: 1.13 GB
⏳ الدفعة 8 - الحجم الحالي: 1.29 GB
⏳ الدفعة 9 - الحجم الحالي: 1.45 GB
⏳ الدفعة 10 - الحجم الحالي: 1.62 GB
⏳ الدفعة 11 - الحجم الحالي: 1.78 GB
⏳ الدفعة 12 - الحجم الحالي: 1.94 GB
⏳ الدفعة 13 - الحجم الحالي: 2.10 GB
⏳ الدفعة 14 - الحجم الحالي: 2.26 GB
⏳ الدفعة 15 - الحجم الحالي: 2.43 GB
⏳ الدفعة 16 - الحجم الحالي: 2.59 GB
⏳ الدفعة 17 - الحجم الحالي: 2.75 GB
⏳ الدفعة 18 - الحجم الحالي: 2.91 GB
⏳ الدفعة 19 - الحجم الحالي: 3.07 GB
⏳ الدفعة 20 - الحجم الحالي: 3.23 GB
⏳ الدفعة 21 - الحجم الحالي: 3.40 GB
⏳ الدفعة 22 - الحجم الحالي: 3.56 GB
⏳ الدفعة 23 - الحجم الحالي: 3.72 GB
⏳ الدفعة 24 - الحجم الحالي: 3.88 GB
⏳ الدفعة 25 - الحجم الحالي: 4.04 GB
⏳ الدفعة 26 - الحجم الحالي: 4.20 GB
⏳ الدفعة 27 - الحجم الحالي: 4.37 GB
⏳ الد

In [35]:
import pandas as pd
import time
import psutil
import os

csv_path = "/content/real_5gb_data.csv"

start_time = time.time()
chunk_size = 50000
chunks = []
process = psutil.Process(os.getpid())

print("🚀 بدء القراءة على دفعات...")

for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunk_size)):
    chunks.append(len(chunk))
    print(f"✅ دفعة {i+1}: {len(chunk)} صفوف | الذاكرة الحالية: {process.memory_info().rss / 1024 ** 2:.2f} MB")

print("\n✅ انتهت عملية القراءة على دفعات")
print("📊 مجموع الصفوف المقروءة:", sum(chunks))
print("⏱ الوقت المستغرق:", round(time.time() - start_time, 2), "ثانية")


🚀 بدء القراءة على دفعات...
✅ دفعة 1: 50000 صفوف | الذاكرة الحالية: 382.45 MB
✅ دفعة 2: 50000 صفوف | الذاكرة الحالية: 390.05 MB
✅ دفعة 3: 50000 صفوف | الذاكرة الحالية: 391.33 MB
✅ دفعة 4: 50000 صفوف | الذاكرة الحالية: 392.10 MB
✅ دفعة 5: 50000 صفوف | الذاكرة الحالية: 392.10 MB
✅ دفعة 6: 50000 صفوف | الذاكرة الحالية: 392.10 MB
✅ دفعة 7: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 8: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 9: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 10: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 11: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 12: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 13: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 14: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 15: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 16: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 17: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 18: 50000 صفوف | الذاكرة الحالية: 392.24 MB
✅ دفعة 19: 50000 صفوف | الذاكرة الحالية: 391.24 MB
✅ دفعة 20: 50

In [36]:
import dask.dataframe as dd
import time
import psutil
import os

csv_path = "/content/real_5gb_data.csv"

start_time = time.time()

print("🚀 تحميل الملف بواسطة Dask...")
df = dd.read_csv(csv_path)

print("✅ تم إنشاء Dask DataFrame")
print("🔍 بعض المعلومات:")
print(df.head())


print("\n📊 حساب الإحصاءات الأساسية (describe)...")
summary = df.describe().compute()

print(summary)
print("\n⏱ الوقت المستغرق:", round(time.time() - start_time, 2), "ثانية")
print("💾 الذاكرة الحالية:", psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2, "MB")


🚀 تحميل الملف بواسطة Dask...
✅ تم إنشاء Dask DataFrame
🔍 بعض المعلومات:
   id  feature1  feature2    feature3  feature4  feature5  feature6  feature7  \
0   0  0.596368  1.204981  157.485217     47454 -1.856000  2.043281  1.536866   
1   1 -0.039102  1.357298  925.315392     13450  1.287307  0.951003  1.356001   
2   2  1.452985  2.559430  165.761364     59989 -1.769704  0.006050 -0.819106   
3   3 -0.376375  0.683392  129.624937     74748 -0.970765 -2.232825 -1.511932   
4   4 -0.154934  1.399191  380.286034     17279 -0.025051  1.566495  1.157440   

   feature8 category1  category2  \
0  1.963903         D     Medium   
1  0.272476         B       High   
2  0.324720         D       High   
3  0.723220         H  Very_High   
4  1.346971         C       High   

                                         text_field1  \
0  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...   
1  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...   
2  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...   


In [37]:
import gzip
import shutil
import time

csv_path = "/content/real_5gb_data.csv"
compressed_path = "/content/real_5gb_data.csv.gz"

start_time = time.time()
print("🚀 بدء ضغط الملف...")

with open(csv_path, 'rb') as f_in:
    with gzip.open(compressed_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ تم ضغط الملف بنجاح.")
print("⏱ الوقت المستغرق للضغط:", round(time.time() - start_time, 2), "ثانية")


🚀 بدء ضغط الملف...
✅ تم ضغط الملف بنجاح.
⏱ الوقت المستغرق للضغط: 300.47 ثانية


In [38]:
import pandas as pd
import psutil
import time

start_time = time.time()
print("🚀 قراءة الملف المضغوط...")

df_compressed = pd.read_csv(compressed_path, compression='gzip', nrows=100000)

print("✅ تم قراءة العينة من الملف المضغوط.")
print("📊 حجم العينة:", df_compressed.shape)
print("⏱ الوقت:", round(time.time() - start_time, 2), "ثانية")
print("💾 الذاكرة:", psutil.Process().memory_info().rss / 1024 ** 2, "MB")


🚀 قراءة الملف المضغوط...
✅ تم قراءة العينة من الملف المضغوط.
📊 حجم العينة: (100000, 18)
⏱ الوقت: 0.93 ثانية
💾 الذاكرة: 518.04296875 MB


#  **مقارنة طرق معالجة ملف 5GB - النتائج**

##  **جدول المقارنة**

| الطريقة | الوقت | الذاكرة | الصفوف |
|---------|--------|----------|---------|
| **Pandas + Chunking** | 82.67 ثانية | 391.26 MB | 9,300,000 |
| **Dask** | 99.66 ثانية | 519.03 MB | 9,300,000 |
| **Compression** | 301.40 ثانية | 518.04 MB | 100,000 |

## 📈 **الترتيب حسب الأداء**

###  **الأسرع: Pandas + Chunking**
- الوقت: 82.67 ثانية
- الذاكرة: 391.26 MB
- كامل البيانات: 9.3 مليون صف

###  **المركز الثاني: Dask**
- الوقت: 99.66 ثانية
- الذاكرة: 519.03 MB
- كامل البيانات: 9.3 مليون صف

###  **المركز الثالث: Compression**
- الوقت الإجمالي: 301.40 ثانية
- الذاكرة: 518.04 MB
- عينة محدودة: 100,000 صف فقط

##  **الاستنتاجات**

1. **Pandas + Chunking الأفضل لملفات 5GB**
2. **Dask مناسب لتحليل البيانات الضخمة**
3. **Compression غير عملي للمعالجة اليومية**
4. **Pandas + Chunking يوازن بين السرعة والذاكرة**
##  **ملاحظة**
- **Dask استهلك حجما اكثر من الرام لأنه يحسب الإحصاءات ويحتفظ ببعض البيانات مؤقتًا**